<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/GraphCodeBERT_1_N_%ED%95%98%EC%9D%B4%ED%8D%BC%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install optuna
!pip install transformers
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_

In [7]:
import numpy as np
import torch
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import optuna
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import re
from transformers import DataCollatorWithPadding

In [42]:
# 데이터 로드 및 전처리
df = pd.read_csv('/content/drive/MyDrive/MSR+julite_Dive_final_train/MSR+julite+Div_final_train.csv')

In [43]:
# 필요한 컬럼만 남기고 제거
df = df[['code', 'labels']]

In [44]:
print(df.columns)

Index(['code', 'labels'], dtype='object')


In [29]:
def preprocess(df, file_name):
    preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE), "", regex=True)  # 주석 한 줄
    preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL), "", regex=True)  # 주석 여러 줄
    preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE), "\n", regex=True)  # 다중개행 한번으로
    preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE), "\t", regex=True)  # tab 변환
    preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE), " ", regex=True)  # 공백 여러개 변환
    preprocess_df.to_csv(file_name, index=False)

In [30]:
def tokenized(examples):
    return tokenizer(examples['code'], padding=True, truncation=True, return_token_type_ids=True)

In [31]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/graphcodebert", num_labels = 10)
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
batch_size = 16
epoch_num = 1000
MAX_LEN = 512
learning_rate = 2e-5

In [34]:
# 데이터셋 로딩
preprocess(df, "preprocess.csv")
dataset = load_dataset("csv", data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized, remove_columns=['code'], batched=True)
encoded_dataset = encoded_dataset.train_test_split(0.3, seed=100)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/15638 [00:00<?, ? examples/s]

In [35]:
# 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [36]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained("microsoft/graphcodebert-base", num_labels=10)

In [37]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    return {"precision": precision, "recall": recall, "f1": f1}

In [38]:
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 3, 10),
        "seed": trial.suggest_int("seed", 1, 40),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
    }

In [39]:
# 트레이닝 아규먼트 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    report_to="none"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
# 트레이너 설정
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=encoded_dataset['train'],
    eval_dataset=encoded_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
# Optuna와 트레이너 통합
best_run = trainer.hyperparameter_search(direction="maximize", n_trials=10, hp_space=hp_space)

[I 2024-05-19 07:36:59,058] A new study created in memory with name: no-name-8228a3fa-3725-40b3-b7f9-a74b4e7af4ab
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


[W 2024-05-19 07:37:37,440] Trial 0 failed with parameters: {'learning_rate': 2.2707459760271932e-05, 'num_train_epochs': 9, 'seed': 13, 'per_device_train_batch_size': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/integrations/integration_utils.py", line 210, in _objective
    trainer.train(resume_from_checkpoint=checkpoint, trial=trial)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1885, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2216, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 3241, in training_step
    torch.cuda.empty_cache()
 

KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best run:")
print(best_run)